In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import torch
import torch.utils.data as Data
import torch.nn.functional as F
import matplotlib.pyplot as plt

torch.manual_seed(1)

LR=0.01
BATCH_SIZE=32
EPOCH=12

x=torch.unsqueeze(torch.linspace(-1, 1, 1000), dim=1)
y=x.pow(2)+0.1*torch.normal(torch.zeros(*x.size()))

plt.scatter(x.numpy(), y.numpy())
plt.show()

torch_dataset=Data.TensorDataset(x, y)
loader=Data.DataLoader(dataset=torch_dataset, 
                       batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

In [ ]:
# net=torch.nn.Sequential(
#     torch.nn.Linear(1, 20), 
#     torch.nn.ReLU(),
#     torch.nn.Linear(20, 1)
# )

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden=torch.nn.Linear(1, 20)
        self.predict=torch.nn.Linear(20, 1)
        
    def forward(self, x):
        x=F.relu(self.hidden(x))
        x=self.predict(x)
        return x
    
net_SGD=Net()
net_Momentum=Net()
net_RMSprop=Net()
net_Adam=Net()

nets=[net_SGD, net_Momentum, net_RMSprop, net_Adam]

In [ ]:
opt_SGD=torch.optim.SGD(net_SGD.parameters(), lr=LR)
opt_Momentum=torch.optim.SGD(net_Momentum.parameters(), lr=LR, momentum=0.8)
opt_RMSprop=torch.optim.RMSprop(net_RMSprop.parameters(), lr=LR, alpha=0.9)
opt_Adam=torch.optim.Adam(net_Adam.parameters(), lr=LR, betas=(0.9, 0.99))

optimizers=[opt_SGD, opt_Momentum, opt_RMSprop, opt_Adam]

loss_func=torch.nn.MSELoss()
losses_his=[[], [], [], []]

In [ ]:
for epoch in range(EPOCH):
    print('Epoch:', epoch)
    
    for step, (b_x, b_y) in enumerate(loader):
        
        for net, opt, l_his in zip(nets, optimizers, losses_his):
            
            output=net(b_x)
            loss=loss_func(output, b_y)
            
            opt.zero_grad()
            loss.backward()
            opt.step()
            
            l_his.append(loss.data.numpy())

In [ ]:
print(len(losses_his), len(losses_his[3]))
print(losses_his[1][1], type(losses_his[1][1]))

In [ ]:
labels=['SGD', 'Momentum', 'RMSprop', 'Adam']

for i, i_his in enumerate(losses_his):
    plt.plot(l_his, label=labels[i])

plt.legend(loc='best')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.ylim((0, 0.15))
plt.show()